In [11]:
from EntityDataset import TrainDataset
from LoadData import LoadData
import pandas as pd
import numpy as np

In [12]:

sentiments = ["Very Negative", "Negative", "Neutral", "Positive", "Very Positive"]
main_roles = ["Protagonist", "Antagonist", "Innocent"]
sentiment_to_main_role = {
    "Very Negative": "Antagonist",
    "Negative": "Antagonist",
    "Neutral": "Innocent",
    "Positive": "Protagonist",
    "Very Positive": "Protagonist"
}
languages = ["EN", "RU", "PT", "HI", "BG"]
base_dir = "train"
txt_file = "subtask-1-annotations.txt"

ld = LoadData()

results_df = pd.DataFrame(columns = ['Language', 'Accuracy', 'Precision_Protagonist', 'Precision_Antagonist', 'Precision_Innocent', 'Recall_Protagonist', 'Recall_Antagonist', 'Recall_Innocent'])

for lang_idx, lang in enumerate(languages):

    total_instances = 0
    match = 0
    # Load data for each language
    data = ld.load_data(base_dir, txt_file, lang)
    train_dataset = TrainDataset(data, base_dir, language=lang, return_sentiment=True, coref=False)
    
    df = pd.DataFrame(columns=['article_id', 'entity_mention', 'main_role', 'predicted_main_role'])

    for idx, sample in enumerate(train_dataset):

        index = np.argmax(sample['sent_sent'])

        sent_main_role = sentiment_to_main_role[sentiments[index]]
        true_main_role = sample["main_role"]

        if sent_main_role == true_main_role:
            match += 1
        
        total_instances += 1

        df = pd.concat([df, pd.DataFrame([{
            'article_id': sample['article_id'],
            'entity_mention': sample['entity_mention'],
            'main_role': true_main_role,
            'predicted_main_role': sent_main_role
        }])], ignore_index=True)
        
    accuracy = match / total_instances

    # Calculate precision and recall for each main role
    precision_dict = {}
    recall_dict = {}

    for role in main_roles:
        true_positives = len(df[(df['main_role'] == role) & (df['predicted_main_role'] == role)])
        predicted_positives = len(df[df['predicted_main_role'] == role])
        actual_positives = len(df[df['main_role'] == role])
        
        precision = true_positives / predicted_positives if predicted_positives > 0 else 0
        recall = true_positives / actual_positives if actual_positives > 0 else 0
        
        precision_dict[role] = precision
        recall_dict[role] = recall

    # Create a row for the CSV
    results_df = pd.concat([results_df,pd.DataFrame({
        'Language': [lang],
        'Accuracy': [accuracy],
        **{f'Precision_{role}': [precision_dict[role]] for role in main_roles},
        **{f'Recall_{role}': [recall_dict[role]] for role in main_roles}
        
    })], ignore_index= True)
    
    # Append to CSV file, create if doesn't exist
    

    df.to_csv(f"./sentiment_res/{lang}_sentiment.csv", index=False)
results_df.to_csv('./sentiment_res/evaluation.csv', header=not pd.io.common.file_exists('./sentiment_res/sentiment.csv'), index=False)

2025-02-26 02:18:33 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-02-26 02:18:33 INFO: Downloaded file to /Users/sankalpa/stanza_resources/resources.json
2025-02-26 02:18:33 WARNING: Language en package default expects mwt, which has been added
2025-02-26 02:18:35 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| pos       | combined_charlm           |
| lemma     | combined_nocharlm         |
| coref     | udcoref_xlm-roberta-lora  |
| depparse  | combined_charlm           |
| ner       | ontonotes-ww-multi_charlm |

2025-02-26 02:18:35 INFO: Using device: cpu
2025-02-26 02:18:35 INFO: Loading: tokenize
2025-02-26 02:18:35 INFO: Loading: mwt
2025-02-26 02:18:35 INFO: Loading: pos
2025-02-26 02:18:37 INFO: Loading: lemma
2025-02-26 02:18:37 INFO: Loading: coref
/opt/homebrew/Caskroom/miniconda/base/envs/semeval/lib/python3.9/site-packages/huggingface_hub/file_download.py:79

2025-02-26 02:19:12 INFO: Downloaded file to /Users/sankalpa/stanza_resources/resources.json
2025-02-26 02:19:14 INFO: Loading these models for language: ru (Russian):
| Processor | Package                  |
----------------------------------------
| tokenize  | syntagrus                |
| pos       | syntagrus_charlm         |
| lemma     | syntagrus_nocharlm       |
| coref     | udcoref_xlm-roberta-lora |
| depparse  | syntagrus_charlm         |
| ner       | wikiner                  |

2025-02-26 02:19:14 INFO: Using device: cpu
2025-02-26 02:19:14 INFO: Loading: tokenize
2025-02-26 02:19:14 INFO: Loading: pos
2025-02-26 02:19:16 INFO: Loading: lemma
2025-02-26 02:19:19 INFO: Loading: coref
/opt/homebrew/Caskroom/miniconda/base/envs/semeval/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downl